In [1]:
import earthaccess
import xarray as xr
import hvplot.xarray
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math
import pandas as pd

In [2]:
auth = earthaccess.login(persist=True)

In [3]:
tspan = ("2024-03-01", "2024-07-31")
bbox = (113.338953078, -43.6345972634, 153.569469029, -10.6681857235)

In [17]:
results = earthaccess.search_data(
    short_name="PACE_OCI_L3M_LANDVI",
    temporal=tspan,
    # bounding_box=bbox,
    granule_name="*.MO.*0p1deg*",  # Daily, 8-day or monthly: Day, 8D or MO | Resolution: 0p1deg or 0.4km
)

In [18]:
paths = earthaccess.open(results)
# paths

QUEUEING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/5 [00:00<?, ?it/s]

In [19]:
ds = xr.open_mfdataset(paths,
    combine="nested",
    concat_dim="date"
                                )

In [20]:
min_lon, max_lat, max_lon, min_lat = bbox
ds_australia = ds.sel(lat=slice(min_lat, max_lat), lon=slice(min_lon, max_lon))

In [21]:
ds_australia["mari"] = ds_australia["mari"].clip(
    min=1.3,
    max=2.0
)
ds_australia["cire"] = ds_australia["cire"].clip(
    min=0.5,
    max=2.0
)
ds_australia["car"] = ds_australia["car"].clip(
    min=1.3,
    max=6.2
)

In [22]:
ds_australia = ds_australia.drop_vars(
        ["palette", "ndvi", "evi", "ndwi", "ndii", "cci", "ndsi", "pri"])
ds_australia

<xarray.Dataset> Size: 8MB
Dimensions:  (date: 5, lat: 329, lon: 403)
Coordinates:
  * lat      (lat) float32 1kB -10.75 -10.85 -10.95 ... -43.35 -43.45 -43.55
  * lon      (lon) float32 2kB 113.4 113.5 113.6 113.7 ... 153.4 153.5 153.6
Dimensions without coordinates: date
Data variables:
    cire     (date, lat, lon) float32 3MB dask.array<chunksize=(1, 17, 139), meta=np.ndarray>
    car      (date, lat, lon) float32 3MB dask.array<chunksize=(1, 17, 139), meta=np.ndarray>
    mari     (date, lat, lon) float32 3MB dask.array<chunksize=(1, 17, 139), meta=np.ndarray>
Attributes: (12/62)
    product_name:                      PACE_OCI.20240301_20240331.L3m.MO.LAND...
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    cdm_data_type:                     grid
    identifier_product_doi_authority:  http://dx.doi.org
    identifier_product_doi:            10.5067/PACE/OCI/L3M/LANDVI/3.0
    data_bins:                         1714795
    data_minimum:                      -108.14531
    data_maximum:                      89.105576

In [23]:
ds_australia_norm = ds_australia.astype(np.float64)
ds_australia_norm = (
    (ds_australia - ds_australia.min())
    / (ds_australia.max() - ds_australia.min())
)

In [24]:
date_values = pd.date_range(start=tspan[0], periods=ds_australia_norm.dims["date"], freq="M")
ds_australia_norm = ds_australia_norm.assign_coords(date=date_values)
ds_australia_norm

<xarray.Dataset> Size: 8MB
Dimensions:  (lat: 329, lon: 403, date: 5)
Coordinates:
  * lat      (lat) float32 1kB -10.75 -10.85 -10.95 ... -43.35 -43.45 -43.55
  * lon      (lon) float32 2kB 113.4 113.5 113.6 113.7 ... 153.4 153.5 153.6
  * date     (date) datetime64[ns] 40B 2024-03-31 2024-04-30 ... 2024-07-31
Data variables:
    cire     (date, lat, lon) float32 3MB dask.array<chunksize=(1, 17, 139), meta=np.ndarray>
    car      (date, lat, lon) float32 3MB dask.array<chunksize=(1, 17, 139), meta=np.ndarray>
    mari     (date, lat, lon) float32 3MB dask.array<chunksize=(1, 17, 139), meta=np.ndarray>

In [25]:
ds = ds_australia_norm
ds

<xarray.Dataset> Size: 8MB
Dimensions:  (lat: 329, lon: 403, date: 5)
Coordinates:
  * lat      (lat) float32 1kB -10.75 -10.85 -10.95 ... -43.35 -43.45 -43.55
  * lon      (lon) float32 2kB 113.4 113.5 113.6 113.7 ... 153.4 153.5 153.6
  * date     (date) datetime64[ns] 40B 2024-03-31 2024-04-30 ... 2024-07-31
Data variables:
    cire     (date, lat, lon) float32 3MB dask.array<chunksize=(1, 17, 139), meta=np.ndarray>
    car      (date, lat, lon) float32 3MB dask.array<chunksize=(1, 17, 139), meta=np.ndarray>
    mari     (date, lat, lon) float32 3MB dask.array<chunksize=(1, 17, 139), meta=np.ndarray>

In [27]:
import xarray as xr
import holoviews as hv
import hvplot.xarray
from holoviews import streams

hv.extension('bokeh')

# === Assume ds is already loaded ===
# ds: xarray.Dataset with dims (date, lat, lon)
# and variables: 'mari', 'cire', 'car'

variables = ['mari', 'cire', 'car']

# Get midpoint for dummy starting point
xmid = float(ds.lon.values[len(ds.lon) // 2])
ymid = float(ds.lat.values[len(ds.lat) // 2])

# --- Create an RGB map ---
# Normalize each band for RGB visualization
def normalize_band(band):
    b = band.isel(date=0)
    return (b - b.min()) / (b.max() - b.min())

r = normalize_band(ds['car'])
g = normalize_band(ds['cire'])
b = normalize_band(ds['mari'])

rgb = hv.RGB((ds.lon.values, ds.lat.values, r.values, g.values, b.values), vdims=['R', 'G', 'B']).opts(
    tools=['hover'], width=500, height=500, xlabel='Longitude', ylabel='Latitude'
)

# --- Set up interactive drawing ---
POINT_LIMIT = 10
color_cycle = hv.Cycle('Category20')
colors = [color_cycle[i] for i in range(5)]

init_point = ([xmid], [ymid], [0])
points = hv.Points(init_point, vdims='id')

points_stream = streams.PointDraw(
    data=points.columns(),
    source=points,
    drag=True,
    num_objects=POINT_LIMIT,
    styles={'fill_color': color_cycle.values[1:POINT_LIMIT+1], 'line_color': 'gray'}
)

# --- Click spectra plot ---
def click_spectra(data):
    coordinates = []
    if data is None or not any(len(d) for d in data.values()):
        coordinates.append((xmid, ymid))
    else:
        coordinates = [c for c in zip(data['x'], data['y'])]

    plots = []
    variables = ['mari', 'cire', 'car']
    color_map = ['blue', 'green', 'red']

    for i, coords in enumerate(coordinates):
        x, y = coords
        data_sel = ds.sel(lon=x, lat=y, method="nearest")

        lines = []
        for var, color in zip(variables, color_map):
            da = data_sel[var]
            da.name = var
            line = da.hvplot.line(
                x='date',
                label=f"{var} ({i})",
                color=color,
                width=400,
                height=250
            )
            lines.append(line)

        overlay = hv.Overlay(lines)
        plots.append(overlay)

        points_stream.data["id"][i] = i

    return hv.Layout(plots).cols(1)


click_dmap = hv.DynamicMap(click_spectra, streams=[points_stream])

# --- Hover spectra plot ---
def hover_spectra(x, y):
    ts = ds.sel(lon=x, lat=y, method='nearest')

    curves = []
    variables = ['mari', 'cire', 'car']
    colors = ['blue', 'green', 'red']

    for var, color in zip(variables, colors):
        da = ts[var]
        da.name = var
        curve = da.hvplot.line(
            x='date', label=var, color=color,
            width=400, height=250
        )
        curves.append(curve)

    return hv.Overlay(curves)

# --- Create hover stream ---
posxy = hv.streams.PointerXY(source=rgb, x=xmid, y=ymid)

hover_dmap = hv.DynamicMap(hover_spectra, streams=[posxy])

# --- Layout ---
layout = (hover_dmap * click_dmap + rgb * points).cols(2).opts(
    hv.opts.Points(
        active_tools=['point_draw'],
        size=10,
        tools=['hover'],
        color='white',
        line_color='gray'
    ),
    hv.opts.Overlay(
        show_legend=True,
        show_title=False,
        fontscale=1.2,
        frame_height=480
    )
)

layout


AttributeError: 'Overlay' object has no attribute group.

:Layout
   .DynamicMap.I :DynamicMap   []
   .Overlay.I    :Overlay
      .RGB.I    :RGB   [x,y]   (R,G,B)
      .Points.I :Points   [x,y]   (id)

In [36]:
ds

<xarray.Dataset> Size: 8MB
Dimensions:  (lat: 329, lon: 403, date: 5)
Coordinates:
  * lat      (lat) float32 1kB -10.75 -10.85 -10.95 ... -43.35 -43.45 -43.55
  * lon      (lon) float32 2kB 113.4 113.5 113.6 113.7 ... 153.4 153.5 153.6
  * date     (date) datetime64[ns] 40B 2024-03-31 2024-04-30 ... 2024-07-31
Data variables:
    cire     (date, lat, lon) float32 3MB dask.array<chunksize=(1, 17, 139), meta=np.ndarray>
    car      (date, lat, lon) float32 3MB dask.array<chunksize=(1, 17, 139), meta=np.ndarray>
    mari     (date, lat, lon) float32 3MB dask.array<chunksize=(1, 17, 139), meta=np.ndarray>

In [41]:
#new

# functions
def click_spectra(data):
    print('click_spectra')
    coordinates = []
    if data is None or not any(len(d) for d in data.values()):
        print('No Data', data)
        coordinates.append(clicked_points[0][0], clicked_points[1][0])
    else:
        print('Data', data)
        coordinates = [c for c in zip(data['x'], data['y'])]
    
    plots_mari = []
    for i, coords in enumerate(coordinates):
        x, y = coords
        data = ds.sel(lon=x, lat=y, method="nearest")
        plots.append(
            data.hvplot.line(
                y="mari",
                x="date",
                color=color_cycle,
                label=f"{i}"
            )
        )
        points_stream.data["id"][i] = i
        
    return hv.Overlay(plots)

def hover_spectra(x,y):
    return ds.sel(lon=x,lat=y,method='nearest').hvplot.line(y='mari',x='date',
                                                                           color='black', frame_width=400)

#rgb = ds.sel(wavelengths=[2150, 860, 670], method='nearest')
rgb = ds.to_dataarray()

mymap = rgb.hvplot.rgb(x='lon', y='lat', bands='variable', aspect = 'equal', frame_height=500)


# Set Point Limit
POINT_LIMIT = 10

# Set up  Color Cycling
color_cycle = hv.Cycle('Category20')
colors = [color_cycle[i] for i in range(5)]

# Get center coordinates of image
xmid = ds.lon.values[int(len(ds.lon) / 2)]
ymid = ds.lat.values[int(len(ds.lat) / 2)]

#
first_point = ([xmid], [ymid], [0])
points = hv.Points(first_point, vdims='id')
points_stream = hv.streams.PointDraw(
    data=points.columns(),
    source=points,
    drag=True,
    num_objects=POINT_LIMIT,
    styles={'fill_color': color_cycle.values[1:POINT_LIMIT+1], 'line_color': 'gray'}
)

posxy = hv.streams.PointerXY(source= mymap, x=xmid, y=ymid)
clickxy = hv.streams.Tap(source= mymap, x=xmid, y=ymid)

# Define the Dynamic Maps
click_dmap = hv.DynamicMap(click_spectra, streams=[points_stream])
hover_dmap = hv.DynamicMap(hover_spectra, streams=[posxy])
print('step x')

# mymap
# # Plot the Map and Dynamic Map side by side
# hv.Layout(hover_dmap*click_dmap + mymap * points).cols(2).opts(
#     hv.opts.Points(active_tools=['point_draw'], size=10, tools=['hover'], color='white', line_color='gray'),
#     hv.opts.Overlay(show_legend=False, show_title=False, fontscale=1.5, frame_height=480)
# )

hv.Layout(hover_dmap + mymap ).cols(2).opts(
    hv.opts.Points(active_tools=['point_draw'], size=10, tools=['hover'], color='white', line_color='gray'),
    hv.opts.Overlay(show_legend=False, show_title=False, fontscale=1.5, frame_height=480)
)

print('finished')

step x
finished


In [42]:
hv.Layout(hover_dmap*click_dmap + mymap * points).cols(2).opts(
    hv.opts.Points(active_tools=['point_draw'], size=10, tools=['hover'], color='white', line_color='gray'),
    hv.opts.Overlay(show_legend=False, show_title=False, fontscale=1.5, frame_height=480)
)

click_spectra
Data {'x': array([133.45001], dtype=float32), 'y': array([-27.150002], dtype=float32), 'id': array([0])}


click_spectra
Data {'x': array([133.45001], dtype=float32), 'y': array([-27.150002], dtype=float32), 'id': array([0]), 'fill_color': ['#aec7e8'], 'line_color': ['g']}


:Layout
   .DynamicMap.I  :DynamicMap   []
      :Overlay
         .Curve.I   :Curve   [date]   (mari)
         .Curve.A_0 :Curve   [date]   (mari)
   .DynamicMap.II :DynamicMap   [date]
      :Overlay
         .RGB.I    :RGB   [lon,lat]   (R,G,B)
         .Points.I :Points   [x,y]   (id)

In [ ]:
mymap = rgb.hvplot.rgb(x='lon', y='lat', bands='variable', aspect = 'equal', frame_height=500)
mymap

In [ ]:
def click_spectra(data):
    coordinates = []
    if data is None or not any(len(d) for d in data.values()):
        coordinates.append(clicked_points[0][0], clicked_points[1][0])
    else:
        coordinates = [c for c in zip(data['lon'], data['lat'])]
    
    plots = []
    for i, coords in enumerate(coordinates):
        x, y = coords
        data = ds.sel(lon=x, lat=y, method="nearest")
        plots.append(
            data.hvplot.line(
                y="mari",
                x="date",
                color=color_cycle,
                label=f"{i}"
            )
        )
        points_stream.data["id"][i] = i
    return hv.Overlay(plots)

import holoviews as hv
from holoviews import streams
hv.extension('bokeh')

# Assume `ds` is already loaded and chunked with Dask
# ds: (variable: 3, date: 378, lat: 329, lon: 403)

# Define variable names
variables = ['mari', 'cire', 'car']

# Extract only those variables
pigment_subset = ds#.sel(variable=variables)

# Get mid-points for default location
xmid = float(ds.lon.values[len(ds.lon) // 2])
ymid = float(ds.lat.values[len(ds.lat) // 2])

mymap = ds.hvplot.rgb(x='lon', y='lat', bands='variables', aspect = 'equal', frame_height=500)

# Interactive drawing
POINT_LIMIT = 10
color_cycle = hv.Cycle('Category20')
colors = [color_cycle[i] for i in range(5)]

# Initial dummy point
init_point = ([xmid], [ymid], [0])
points = hv.Points(init_point, vdims='id')

points_stream = streams.PointDraw(
    data=points.columns(),
    source=points,
    drag=True,
    num_objects=POINT_LIMIT,
    styles={'fill_color': color_cycle.values[1:POINT_LIMIT+1], 'line_color': 'gray'}
)

def hover_spectra(x, y):
    # Extract the pixel time series at the nearest location
    ts = ds.sel(lon=x, lat=y, method='nearest')

    # Create a line for each variable
    curves = []
    colors = ['red', 'green', 'blue']
    variables = ['mari', 'cire', 'car']

    for var, color in zip(variables, colors):
        da = ts.sel(variable=var)
        da.name = var  # Name the DataArray
        curve = da.hvplot.line(
            x='date', label=var, color=color, frame_width=400
        )
        curves.append(curve)

    # Combine all three curves into one overlay
    return curves[0] * curves[1] * curves[2]


posxy = hv.streams.PointerXY(source= mymap, x=xmid, y=ymid)
clickxy = hv.streams.Tap(source= mymap, x=xmid, y=ymid)


click_dmap = hv.DynamicMap(click_spectra, streams=[points_stream])
hover_dmap = hv.DynamicMap(hover_spectra, streams=[posxy])

# Plot the Map and Dynamic Map side by side
hv.Layout(hover_dmap*click_dmap + mymap * points).cols(2).opts(
    hv.opts.Points(active_tools=['point_draw'], size=10, tools=['hover'], color='white', line_color='gray'),
    hv.opts.Overlay(show_legend=False, show_title=False, fontscale=1.5, frame_height=480)
)

In [ ]:
import xarray as xr
import hvplot.xarray
import holoviews as hv
from holoviews import streams

hv.extension('bokeh')

# =====================================
# Dataset setup
# Assume `ds` is already loaded
# Dimensions: (variable: 3, date: 13, lat: 329, lon: 403)
# =====================================

# Define variable names
variables = ['mari', 'cire', 'car']
pigment_subset = ds.sel(variable=variables)

# Select a default date to get a 3D (variable, lat, lon) slice
default_date = str(ds.date.values[0])[:10]  # e.g., '2024-03-31'
rgb_data = pigment_subset.sel(date=default_date)

# Get mid-points for initial dummy point
xmid = float(ds.lon.values[len(ds.lon) // 2])
ymid = float(ds.lat.values[len(ds.lat) // 2])

# Create the RGB image
mymap = rgb_data.hvplot.rgb(
    x='lon',
    y='lat',
    bands='variable',
    aspect='equal',
    frame_height=500,
)

# =====================================
# PointDraw + Spectra Viewer
# =====================================

# Max number of interactive points
POINT_LIMIT = 10
color_cycle = hv.Cycle('Category20')
colors = [color_cycle[i] for i in range(POINT_LIMIT)]

# Initial point to seed PointDraw tool
init_point = ([xmid], [ymid], [0])
points = hv.Points(init_point, vdims='id')

# Setup interactive drawing stream
points_stream = streams.PointDraw(
    data=points.columns(),
    source=points,
    drag=True,
    num_objects=POINT_LIMIT,
    styles={'fill_color': color_cycle.values[1:POINT_LIMIT+1], 'line_color': 'gray'}
)

# Spectra display function
def click_spectra(data):
    coordinates = []
    if data is None or not any(len(d) for d in data.values()):
        coordinates.append((xmid, ymid))
    else:
        coordinates = [c for c in zip(data['x'], data['y'])]

    plots = []
    variables = ['mari', 'cire', 'car']
    color_map = ['blue', 'green', 'red']

    for i, coords in enumerate(coordinates):
        x, y = coords
        data_sel = ds.sel(lon=x, lat=y, method="nearest")

        curves = []
        for var, color in zip(variables, color_map):
            da = data_sel[var]
            da.name = var
            curve = da.hvplot.line(
                x='date',
                label=f"{var} ({i})",
                color=color,
                width=400,
                height=250
            )
            curves.append(curve)

        overlay = hv.Overlay(curves).relabel(f"Location {i}")
        plots.append(overlay)

    return hv.Layout(plots).cols(1)


# Dynamic spectra plot
click_dmap = hv.DynamicMap(click_spectra, streams=[points_stream])

# =====================================
# Final Layout
# =====================================

layout = (click_dmap + (mymap * points)).cols(2).opts(
    hv.opts.Points(
        active_tools=['point_draw'],
        size=10,
        tools=['hover'],
        color='white',
        line_color='gray'
    ),
    hv.opts.Overlay(
        show_legend=True,
        show_title=False,
        fontscale=1.5,
        frame_height=480
    )
)

layout


In [ ]:
import holoviews as hv
from holoviews import streams
hv.extension('bokeh')

# Assume `ds` is already loaded and chunked with Dask
# ds: (variable: 3, date: 378, lat: 329, lon: 403)

# Define variable names
variables = ['mari', 'cire', 'car']

# Extract only those variables
pigment_subset = ds#.sel(variable=variables)

# Get mid-points for default location
xmid = float(ds.lon.values[len(ds.lon) // 2])
ymid = float(ds.lat.values[len(ds.lat) // 2])

In [ ]:
mymap = pigment_subset.hvplot.rgb(x='lon', y='lat', bands='variables', aspect = 'equal', frame_height=500)

In [ ]:
# Interactive drawing
POINT_LIMIT = 10
color_cycle = hv.Cycle('Category20')
colors = [color_cycle[i] for i in range(5)]

# Initial dummy point
init_point = ([xmid], [ymid], [0])
points = hv.Points(init_point, vdims='id')

points_stream = streams.PointDraw(
    data=points.columns(),
    source=points,
    drag=True,
    num_objects=POINT_LIMIT,
    styles={'fill_color': color_cycle.values[1:POINT_LIMIT+1], 'line_color': 'gray'}
)

In [ ]:
def hover_spectra(x, y):
    # Extract the pixel time series at the nearest location
    ts = ds.sel(lon=x, lat=y, method='nearest')

    # Create a line for each variable
    curves = []
    colors = ['red', 'green', 'blue']
    variables = ['mari', 'cire', 'car']

    for var, color in zip(variables, colors):
        da = ts.sel(variable=var)
        da.name = var  # Name the DataArray
        curve = da.hvplot.line(
            x='date', label=var, color=color, frame_width=400
        )
        curves.append(curve)

    # Combine all three curves into one overlay
    return curves[0] * curves[1] * curves[2]


In [ ]:
hover_dmap = hv.DynamicMap(hover_spectra, streams=[posxy])

# Plot the Map and Dynamic Map side by side
hv.Layout(hover_dmap*click_dmap + mymap * points).cols(2).opts(
    hv.opts.Points(active_tools=['point_draw'], size=10, tools=['hover'], color='white', line_color='gray'),
    hv.opts.Overlay(show_legend=False, show_title=False, fontscale=1.5, frame_height=480)
)

In [ ]:
# # Create DynamicMap for clicking
# click_dmap = hv.DynamicMap(click_spectra, streams=[points_stream])

# # Display interactive plot
# layout = click_dmap + points
# layout.cols(1).opts(
#     hv.opts.Points(active_tools=['point_draw'], size=10, tools=['hover'], color='white', line_color='gray'),
#     hv.opts.Overlay(show_legend=False, show_title=False, fontscale=1.2, frame_height=400)
# )